In [11]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from keras import regularizers


In [12]:
# download dataset
import pathlib
dataset_link = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
data_dir = tf.keras.utils.get_file('Images', origin=dataset_link, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
# create dataset
batch_size = 32
img_h = 200
img_w = 200

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_h, img_w),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_h, img_w),
  batch_size=batch_size)

# test dataset
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

# improve performance of dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
# create the model
categories_number = 120

model = Sequential(
    [
     layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_h, img_w, 3)),
     layers.experimental.preprocessing.RandomRotation(0.05),
     #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),

     layers.Conv2D(16, 3, padding='same', kernel_regularizer=regularizers.l2(0.001),activation='relu'),
     layers.MaxPooling2D(),
     
     layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.001),activation='relu'),
     layers.MaxPooling2D(),
     
     layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.l2(0.001),activation='relu'),
     layers.MaxPooling2D(),

     #layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.001),activation='relu'),
     #layers.MaxPooling2D(),
     #layers.Dropout(0.01),

     #layers.Conv2D(16, 3, padding='same', activation='relu'),
     #layers.MaxPooling2D(),
     #layers.Conv2D(32, 3, padding='same', activation='relu'),
     #layers.MaxPooling2D(),
     #layers.Conv2D(64, 3, padding='same', activation='relu'),
     #layers.MaxPooling2D(),

     # fully connected layers
     layers.Flatten(),
     layers.Dense(128, activation='relu'),
     #layers.Dense(categories_number, activation = 'softmax')
     layers.Dense(categories_number)
    ]
)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# train the model
epochs = 15

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)